# ResNet
The ResNet architecture, winner of the 2015 ImageNet ILSVRC contest for image classification, was one of the first to incorporate the conventional steps of both maximizing coarse-level feature extraction and minimizing parameters with feature map reduction. When comparing their model to VGG, the authors of ResNet found they could reduce the size of the extracted feature maps by a whopping 94% in the stem component, reducing memory footprint and increasing the model performance without affecting the accuracy

**NOTE** The process of comparing a newer model to the previous SOTA model is called an *ablation study*, and is common practice in the machine learning field. Basically, the researchers replicate the study of the previous model, and then use the same configuration (say, image augmentation or learning rate) for their new model. This allows them to make direct apple-to-apple comparisons with the earlier models

The ResNet authors also chose to use an extremely large coarse filter of size 7 × 7, which covered an area of 49 pixels. Their reasoning here was that the model needed a very
large filter to be effective. The drawback was a substantial increase in matrix multiply, or matmul, operations in the stem component. Eventually, researchers subsequently
found in later SOTA models a 5 × 5 filter to be as effective and more efficient. In conventional CNNs, the 5 × 5 filter is generally replaced with a stack of two 3 × 3 filters, with the first convolution being unstrided (no pooling) and the second convolution being strided (with feature pooling).

For several years, the ResNet v1 and refined v2 became the de facto architecture used in production for image classification, and the backbone in object-detection models.
Beyond its improved performance and accuracy, public versions of pretrained ResNets for image classification, object detection, and image segmentation tasks were widely
available, so this architecture became the standard for transfer learning as well. Even today, in high-profile model zoos, like TensorFlow Hub, pretrained ResNet v2 continues
to be highly prevalent as the image-classification backbone. The more modern convention today for a pretrained image classification, however, is the smaller, faster, and more accurate EfficientNet. Figure 5.4 depicts the layers in a ResNet stem component.

<img src="img_1.png"/>

In ResNet, the stem component consists of one convolutional layer for coarse feature extraction. The model uses a 7 × 7 filter size to obtain coarse features over a wider window, under the theory it would extract larger features. The 7 × 7 filter covers 49 pixels (in contrast to a 3 × 3, which covers 9 pixels). Using a much larger filter size also substantially increases the number of computations (matrix multiplies) per filter step (as the filter is slid across the image). On a per pixel basis, the 3 × 3 has 9 matrix multiplications, and the 7 × 7 has 49. After ResNet, the convention of using 7 × 7 to obtain larger coarse-level features was not pursued anymore.

 Note that both the VGG and ResNet stem components output 64 initial feature maps. This continues to be a fairly common convention, learned through trial and
error by researchers.

 For feature map reduction, the ResNet stem group does both a feature pooling step (strided convolution) and downsampling (max pooling).

The convolutional layer uses no padding when sliding the filter across the image. Thus, when the filter reaches the edge of the image, it stops. Since the last pixels
before an edge do not have their own slide by the filter, the output size is smaller than the input size, as depicted in figure 5.5. The consequence is that the size of the input and output feature maps are not preserved. For example, in a convolution of stride 1, filter size of 3 × 3, and input feature map size 32 × 32, the output feature maps will be 30 × 30. Calculating the loss in size is straightforward. If the filter size is N × N, the loss in size will be N – 1 pixels. In TF.Keras, this is specified with the keyword parameter padding='valid' to the Conv2D layer.

<img src="img_2.png" />

Alternatively, we can slide the filter over the edge until the last row and column are covered. But part of the filter would hang over imaginary pixels. This way, the last pixel before an edge have their own slide by the filter, and the size of the output feature map is preserved.

Several strategies exist for padding the imaginary pixels. The most common convention today is to pad the imaginary pixels with the same pixel value at the edge,
as depicted in figure 5.5. In TF.Keras, this is specified with the keyword parameter padding='same' to the Conv2D layer.

 ResNet predated this convention and padded the imaginary pixels with zero values; this is why you see in the stem group the ZeroPadding2D layers, where a zero padding is placed around the image. Today we typically pad the image with the same padding and defer feature map size reduction to pooling or feature pooling. Through trial and error, researchers learned that this approach gave a better result in maintaining feature extraction information at the edge of the image.

<img src="img_3.png"/>

Figure 5.7 shows a convolution with padding on an image of size H × W × 3 (three channels for RGB) with multiple filters C. Here, we would output a feature map of size
H × W × C.

<img src="img_4.png">

Would you ever see a stem convolution with only a single outputted feature map, as depicted in figure 5.6? The answer is no. That’s because a single filter can learn to
extract only a single coarse feature. That’s not going to work for images! Even if our images are simple sequences of parallel lines (a single feature) and we just want to
count the lines, it still won’t work: we can’t control which feature a filter will learn to extract. A certain amount of randomness remains in the process, so we need some
redundancy to guarantee that enough of the filters will learn to extract the important features.
 Would you ever output a single feature map, somewhere else in a CNN? The answer is yes. That would be an aggressive reduction by a 1 × 1 bottleneck convolution. A 1 × 1 bottleneck convolution is typically used for feature reuse between different convolutions in a CNN.

Once again, it involves a tradeoff. On the one hand, you want to combine the benefits of feature extraction/learning at one place in the CNN with another place in the
CNN (feature reuse). The problem is that reusing the entire previous feature maps, in number and size, would create a potential explosion in parameters. That resulting
increase in memory footprint and reduction in speed offsets the benefit. The ResNet authors settled on the amount of feature reduction as the best tradeoff between accuracy on the one hand, and size and performance on the other.

Next, take a look at a sample for coding the ResNet stem component in the Idiomatic design pattern. The code demonstrates the sequential flow through the layers
illustrated previously in figure 5.3 for the stem:
 The Conv2D layer uses 7 × 7 filter size for coarse-level feature extraction and strides=(2, 2) for feature pooling.
 MaxPooling layers perform downsampling for further feature map reduction.

<img src="img_5.png">

In [1]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ReLU
def stem(inputs):
    """Construct a stem convolutional Group"""
    outputs = Conv2D(64, kernel_size=(7, 7), strides=(1, 1), padding="same")(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = ReLU()(outputs)
    outputs = MaxPooling2D((3,3), strides=(2,2), padding="same")(outputs)
    return outputs